In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir = f"datasets/friedman/many"
results_dir_tanh = "results/regression/single_layer/tanh/friedman"

#model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh", "Beta Horseshoe tanh", "Beta Student T tanh"]
model_names_tanh_nodewise = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh nodewise", "Dirichlet Student T tanh nodewise"]#, "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise"]

#tanh_fits = {}
tanh_fits_nodewise = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    
    # tanh_fit = get_model_fits(
    #     config=full_config_path,
    #     results_dir=results_dir_tanh,
    #     models=model_names_tanh,
    #     include_prior=False,
    # )
    tanh_fit_nodewise = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh_nodewise,
        include_prior=False,
    )

    #tanh_fits[base_config_name] = tanh_fit  # use clean key
    tanh_fits_nodewise[base_config_name] = tanh_fit_nodewise  # use clean key

In [ ]:
data_dir = f"datasets/friedman"
results_dir_tanh = "results/regression/single_layer/tanh/friedman"

#model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh", "Dirichlet Student T tanh", "Beta Horseshoe tanh", "Beta Student T tanh"]
model_names_tanh_nodewise = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh nodewise", "Dirichlet Student T tanh nodewise"]#, "Beta Horseshoe tanh nodewise", "Beta Student T tanh nodewise"]

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    
    # tanh_fit = get_model_fits(
    #     config=full_config_path,
    #     results_dir=results_dir_tanh,
    #     models=model_names_tanh,
    #     include_prior=False,
    # )
    tanh_fit_nodewise = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh_nodewise,
        include_prior=False,
    )

    #tanh_fits[base_config_name] = tanh_fit  # use clean key
    tanh_fits_nodewise[base_config_name] = tanh_fit_nodewise  # use clean key

## OLD SPARSITY

In [6]:
from utils.sparsity import forward_pass_relu, forward_pass_tanh, local_prune_weights
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from utils.generate_data import sample_gaussian_copula_uniform

def generate_Friedman_data_v2(N=100, D=10, sigma=1.0, test_size=0.2, seed=42, standardize_y=True, return_scale=True):
    np.random.seed(seed)
    X = np.random.uniform(0, 1, size=(N, D))
    x0, x1, x2, x3, x4 = X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4]

    y_clean = (
        10 * np.sin(np.pi * x0 * x1) +
        20 * (x2 - 0.5) ** 2 +
        10 * x3 +
        5.0 * x4
    )
    y = y_clean + np.random.normal(0, sigma, size=N)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

    if not standardize_y:
        return (X_train, X_test, y_train, y_test) if not return_scale else (X_train, X_test, y_train, y_test, 0.0, 1.0)

    y_mean = y_train.mean()
    y_std = y_train.std() if y_train.std() > 0 else 1.0

    y_train_s = (y_train - y_mean) / y_std
    y_test_s = (y_test - y_mean) / y_std

    if return_scale:
        return X_train, X_test, y_train_s, y_test_s, y_mean, y_std
    return X_train, X_test, y_train_s, y_test_s

def make_large_eval_set(
    generator_fn,
    N_train,
    D,
    sigma,
    seed,
    n_eval=5000,
    standardize_y=True
):
    """
    Returns X_eval, y_eval (standardized if standardize_y=True), plus y_mean,y_std
    defined from the training split.
    """
    N_total = N_train + n_eval

    X_tr, X_te, y_tr, y_te, y_mean, y_std = generator_fn(
        N=N_total, D=D, sigma=sigma, test_size=n_eval / N_total, seed=seed,
        standardize_y=standardize_y, return_scale=True
    )
    # Now X_te has approx n_eval points (exact given test_size construction).
    return X_te, np.asarray(y_te).squeeze(), y_mean, y_std

def compute_sparse_rmse_results(seeds, all_seeds, models, all_fits, get_N_sigma, forward_pass, folder,
                         sparsity=0.0, prune_fn=None):
    #results = []
    posterior_means = []
    def choose_gen(folder):
        if folder == "friedman":
            return generate_Friedman_data_v2
        elif folder == "friedman/many":
            return generate_Friedman_data_v2
        else: 
            print("Error")
            return None

    for seed in all_seeds:
        N, sigma = get_N_sigma(seed, seeds)
        dataset_key = f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}'

        # Build large eval set consistent with training split standardization
        gen_fn = choose_gen(folder)
        X_test, y_test, y_mean, y_std = make_large_eval_set(
            generator_fn=gen_fn,
            N_train=N,
            D=10,
            sigma=sigma,
            seed=seed,
            n_eval=500,
            standardize_y=True
        )
        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                W1_samples = fit.stan_variable("W_1")           # (S, P, H)
                W2_samples = fit.stan_variable("W_L")           # (S, H, O)
                b1_samples = fit.stan_variable("hidden_bias")   # (S, O, H)
                b2_samples = fit.stan_variable("output_bias")   # (S, O)
            except KeyError:
                print(f"[SKIP] Model or posterior not found: {dataset_key} -> {model}")
                continue

            S = W1_samples.shape[0]
            #rmses = np.zeros(S)
            #print(y_test.shape)
            y_hats = np.zeros((S, y_test.shape[0]))

            for i in range(S):
                W1 = W1_samples[i]
                W2 = W2_samples[i]

                # Apply pruning mask if requested
                if prune_fn is not None and sparsity > 0.0:
                    masks = prune_fn([W1, W2], sparsity)
                    W1 = W1 * masks[0]
                    #W2 = W2 * masks[1]

                y_hat = forward_pass(X_test, W1, b1_samples[i][0], W2, b2_samples[i])
                y_hats[i] = y_hat.squeeze()  # Store the prediction for each sample
                #rmses[i] = np.sqrt(np.mean((y_hat.squeeze() - y_test)**2))
                
            posterior_mean = np.mean(y_hats, axis=0)
            posterior_mean_rmse = np.sqrt(np.mean((posterior_mean - y_test.squeeze())**2))

            posterior_means.append({
                'seed': seed,
                'N': N,
                'sigma': sigma,
                'model': model,
                'sparsity': sparsity,
                'posterior_mean_rmse_orig': posterior_mean_rmse,
                'posterior_mean_rmse': posterior_mean_rmse * y_std
            })

            # for i in range(S):
            #     results.append({
            #         'seed': seed,
            #         'N': N,
            #         'sigma': sigma,
            #         'model': model,
            #         'sparsity': sparsity,
            #         'rmse': rmses[i]
            #     })

    #df_rmse = pd.DataFrame(results)
    df_posterior_rmse = pd.DataFrame(posterior_means)

    return df_posterior_rmse#df_rmse, df_posterior_rmse


sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]
seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
N100_seeds = [1, 3, 4, 5, 6]
N200_seeds = [2, 7, 8, 9, 10]
N500_seeds = [11, 12, 13, 14, 15]
all_seeds = [N100_seeds, N200_seeds, N500_seeds]

def get_N_sigma(seed, all_seeds):
    if seed in all_seeds[0]:
        N=100
    elif seed in all_seeds[1]:
        N=200
    else:
        N=500
    sigma=1.00
    return N, sigma



In [7]:
df_posterior_rmse_tanh_nodewise = {}

for sparsity in sparsity_levels:
    df_posterior_rmse_tanh_nodewise[sparsity] = compute_sparse_rmse_results(
        all_seeds, seeds, model_names_tanh_nodewise, tanh_fits_nodewise, get_N_sigma, forward_pass_tanh, folder = "friedman",
        sparsity=sparsity, prune_fn=local_prune_weights
    )

In [8]:
df_post_tanh_full_nodewise_old = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_posterior_rmse_tanh_nodewise.items()],
    ignore_index=True
)

In [9]:
def aggregate_over_seeds(df: pd.DataFrame, metric_col: str, agg_fun: str = "mean") -> pd.DataFrame:
    """
    Aggregate metric_col across seeds within each (N, sparsity, model, sigma).
    Returns a dataframe with columns:
      N, sparsity, model, sigma, n_seeds, center, lo, hi, sd, se
    where lo/hi depend on SHOW_BAND + BAND_KIND.
    """
    group_cols = ["N", "sparsity", "model"]
    if "sigma" in df.columns:
        group_cols = ["sigma"] + group_cols

    g = df.groupby(group_cols)[metric_col]

    out = g.agg(
        n_seeds="count",
        mean="mean",
        median="median",
        sd="std",
    ).reset_index()

    out["sd"] = out["sd"].fillna(0.0)
    out["se"] = np.where(out["n_seeds"] > 0, out["sd"] / np.sqrt(out["n_seeds"]), np.nan)

    if agg_fun == "median":
        out["center"] = out["median"]
    else:
        out["center"] = out["mean"]

    return out


df_post_tanh_full_nodewise_agg_old = aggregate_over_seeds(df_post_tanh_full_nodewise_old, "posterior_mean_rmse")

## NEW SPARSITY

In [10]:
import re
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

def _find_run_key(fits_by_run, *, N, D, sigma, seed, correlated=False):
    """
    Tries to find the right key inside e.g. tanh_fits for the requested (N, D, sigma, seed).
    Works even if your sigma formatting differs a bit (e.g. 1, 1.0, 1.00).
    """
    # Common naming you showed: Friedman_N100_p10_sigma1.00_seed3
    # If you also have "CorrelatedFriedman" etc, we try both patterns.
    sigma_strs = [
        f"{sigma}",
        f"{sigma:.1f}",
        f"{sigma:.2f}",
        f"{sigma:.3f}",
    ]
    candidates = []
    for s in sigma_strs:
        if correlated:
            candidates.append(f"CorrelatedFriedman_N{N}_p{D}_sigma{s}_seed{seed}")
        candidates.append(f"Friedman_N{N}_p{D}_sigma{s}_seed{seed}")

    for k in candidates:
        if k in fits_by_run:
            return k

    # If exact match not found, do a regex search (robust to extra prefixes/suffixes)
    # Example matches: "...Friedman_N100_p10_sigma1.00_seed3..."
    sig_pat = "|".join(re.escape(s) for s in sigma_strs)
    base = r"Friedman" if not correlated else r"(?:CorrelatedFriedman|Friedman)"
    pat = re.compile(
        rf"{base}_N{N}_p{D}_sigma(?:{sig_pat})_seed{seed}"
    )
    matches = [k for k in fits_by_run.keys() if pat.search(k)]
    if len(matches) == 1:
        return matches[0]
    if len(matches) > 1:
        # Prefer the shortest/most exact-looking key
        matches = sorted(matches, key=len)
        return matches[0]

    raise KeyError(
        f"Could not find a run in fits_by_run for N={N}, D={D}, sigma={sigma}, seed={seed}, correlated={correlated}.\n"
        f"Example expected key: 'Friedman_N{N}_p{D}_sigma{sigma:.2f}_seed{seed}'."
    )

def build_global_mask_from_posterior(
    W_samples,
    sparsity,
    method="Eabs",          # "Eabs" or "Eabs_stability"
    stability_quantile=0.1, # used if method="Eabs_stability"
    prune_smallest=True
):
    """
    W_samples: array (S, ..., ...) posterior draws of a weight matrix.
    sparsity: fraction to prune (q). Keeps (1-q).
    Returns mask with same trailing shape as one draw, dtype float {0,1}.
    """
    assert 0.0 <= sparsity < 1.0
    S = W_samples.shape[0]
    W_abs = np.abs(W_samples)  # (S, ...)

    # Importance score a = E|w|
    a = W_abs.mean(axis=0)     # (..., ...)

    if method == "Eabs":
        score = a
    elif method == "Eabs_stability":
        # Stability proxy pi = P(|w| > t), where t is a small global quantile of |w|
        t = np.quantile(W_abs.reshape(S, -1), stability_quantile)
        pi = (W_abs > t).mean(axis=0)
        # Combine: emphasize both "large on average" and "consistently non-tiny"
        score = a * pi
    else:
        raise ValueError("method must be 'Eabs' or 'Eabs_stability'")

    # Decide how many to prune
    num_params = score.size
    k_prune = int(np.floor(sparsity * num_params))
    if k_prune == 0:
        return np.ones_like(score, dtype=float)

    flat = score.reshape(-1)

    if prune_smallest:
        # prune lowest scores
        thresh = np.partition(flat, k_prune - 1)[k_prune - 1]
        mask = (score > thresh).astype(float)
        # if ties create too many kept/pruned, fix deterministically
        # (rare but possible with many equal scores)
        if mask.sum() > num_params - k_prune:
            # drop some tied-at-threshold entries
            idx_tied = np.where(score.reshape(-1) == thresh)[0]
            need_drop = int(mask.sum() - (num_params - k_prune))
            if need_drop > 0:
                mask_flat = mask.reshape(-1)
                mask_flat[idx_tied[:need_drop]] = 0.0
                mask = mask_flat.reshape(score.shape)
        elif mask.sum() < num_params - k_prune:
            # add some tied entries if we kept too few
            idx_tied = np.where(score.reshape(-1) == thresh)[0]
            need_add = int((num_params - k_prune) - mask.sum())
            if need_add > 0:
                mask_flat = mask.reshape(-1)
                # add back from tied
                add_candidates = idx_tied[mask_flat[idx_tied] == 0.0]
                mask_flat[add_candidates[:need_add]] = 1.0
                mask = mask_flat.reshape(score.shape)
    else:
        # prune largest (not typical)
        thresh = np.partition(flat, num_params - k_prune)[num_params - k_prune]
        mask = (score < thresh).astype(float)

    return mask

def precompute_global_masks(
    all_fits,
    dataset_key,
    model,
    sparsity_levels,
    prune_W2=False,
    method="Eabs_stability"
):
    """
    Returns dict: sparsity -> (mask_W1, mask_W2 or None)
    """
    fit = all_fits[dataset_key][model]["posterior"]

    W1_samples = fit.stan_variable("W_1")  # (S, P, H)
    W2_samples = fit.stan_variable("W_L")  # (S, H, O) or (S, H) depending on O

    masks = {}
    for q in sparsity_levels:
        mask_W1 = build_global_mask_from_posterior(W1_samples, q, method=method)
        mask_W2 = None
        if prune_W2:
            mask_W2 = build_global_mask_from_posterior(W2_samples, q, method=method)
        masks[q] = (mask_W1, mask_W2)
    return masks


In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from utils.generate_data import sample_gaussian_copula_uniform

def make_large_eval_set(
    generator_fn,
    N_train,
    D,
    sigma,
    seed,
    n_eval=5000,
    standardize_y=True
):
    """
    Returns X_eval, y_eval (standardized if standardize_y=True), plus y_mean,y_std
    defined from the training split.
    """
    N_total = N_train + n_eval

    X_tr, X_te, y_tr, y_te, y_mean, y_std = generator_fn(
        N=N_total, D=D, sigma=sigma, test_size=n_eval / N_total, seed=seed,
        standardize_y=standardize_y, return_scale=True
    )
    # Now X_te has approx n_eval points (exact given test_size construction).
    return X_te, np.asarray(y_te).squeeze(), y_mean, y_std

def _logsumexp(a, axis=None):
    amax = np.max(a, axis=axis, keepdims=True)
    out = amax + np.log(np.sum(np.exp(a - amax), axis=axis, keepdims=True))
    return np.squeeze(out, axis=axis)

def gaussian_nll_pointwise(y, mu, sigma):
    return 0.5*np.log(2*np.pi*(sigma**2)) + 0.5*((y-mu)**2)/(sigma**2)

def compute_sparse_metrics_results_globalmask_large_eval(
    seeds, all_seeds, models, all_fits, get_N_sigma, forward_pass,
    folder,
    sparsity=0.0,
    masks_cache=None,
    prune_W2=False,
    compute_nll=True,
    noise_var_name="sigma",
    n_eval=5000,
    D=10,
    standardize_y=True,
    # pass the correct generator functions
    gen_uncorr=None,
    gen_corr=None,
):
    """
    Evaluate on a large generated test set instead of the stored tiny X_test/y_test.
    Assumes model was trained on standardized y if standardize_y=True.
    """
    assert gen_uncorr is not None #and gen_corr is not None, "Pass both generator functions."

    results = []
    posterior_means = []

    # choose generator based on folder name
    def choose_gen(folder):
        return gen_uncorr if "friedman" in folder else gen_corr

    for seed in seeds:
        N, sigma = get_N_sigma(seed, all_seeds)
        dataset_key = f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}'

        # Build large eval set consistent with training split standardization
        gen_fn = choose_gen(folder)
        X_test, y_test, y_mean, y_std = make_large_eval_set(
            generator_fn=gen_fn,
            N_train=N,
            D=D,
            sigma=sigma,
            seed=seed,
            n_eval=n_eval,
            standardize_y=standardize_y
        )

        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                W1_samples = fit.stan_variable("W_1")           # (S, P, H)
                W2_samples = fit.stan_variable("W_L")           # (S, H, O)
                b1_samples = fit.stan_variable("hidden_bias")   # (S, O, H)
                b2_samples = fit.stan_variable("output_bias")   # (S, O)

                noise_samples = None
                if compute_nll:
                    try:
                        noise_samples = fit.stan_variable(noise_var_name).squeeze()
                    except Exception:
                        noise_samples = None
            except KeyError:
                print(f"[SKIP] Model or posterior not found: {dataset_key} -> {model}")
                continue

            S = W1_samples.shape[0]
            y_hats = np.zeros((S, y_test.shape[0]))
            #rmses = np.zeros(S)

            mask_W1 = mask_W2 = None
            if masks_cache is not None and sparsity > 0.0:
                mask_W1, mask_W2 = masks_cache[(dataset_key, model)][sparsity]

            for i in range(S):
                W1 = W1_samples[i]
                W2 = W2_samples[i]

                if mask_W1 is not None:
                    W1 = W1 * mask_W1
                if prune_W2 and (mask_W2 is not None):
                    W2 = W2 * mask_W2

                y_hat = forward_pass(X_test, W1, b1_samples[i][0], W2, b2_samples[i]).squeeze()
                y_hats[i] = y_hat
                #rmses[i] = np.sqrt(np.mean((y_hat - y_test)**2))

            # posterior mean RMSE (standardized scale)
            posterior_mean = y_hats.mean(axis=0)
            posterior_mean_rmse = np.sqrt(np.mean((posterior_mean - y_test)**2))

            out_pm = {
                'seed': seed,
                'N': N,
                'sigma': sigma,
                'model': model,
                'sparsity': sparsity,
                'n_eval': y_test.shape[0],
                'posterior_mean_rmse': posterior_mean_rmse,
                'posterior_mean_rmse_orig': posterior_mean_rmse * y_std,  # back to original y scale
            }

            if compute_nll:
                if noise_samples is None:
                    sig_s = np.ones(S)
                else:
                    sig_s = np.asarray(noise_samples).reshape(-1)[:S]

                # Expected NLL
                nll_draws = np.array([
                    gaussian_nll_pointwise(y_test, y_hats[i], sig_s[i]).mean()
                    for i in range(S)
                ])
                expected_nll = nll_draws.mean()

                # Predictive (mixture) NLL
                loglik = -np.stack([
                    gaussian_nll_pointwise(y_test, y_hats[i], sig_s[i])
                    for i in range(S)
                ], axis=0)  # (S, n_eval)
                lppd = (_logsumexp(loglik, axis=0) - np.log(S)).mean()
                predictive_nll = -lppd

                out_pm["expected_nll"] = expected_nll
                out_pm["predictive_nll"] = predictive_nll

                # Optional: predictive_nll on original scale (only if you also rescale sigma)
                # If your sigma posterior is on standardized scale, original sigma = sig_s * y_std.
                out_pm["predictive_nll_orig"] = predictive_nll + np.log(y_std)  # see note below

            posterior_means.append(out_pm)

            # for i in range(S):
            #     row = {
            #         'seed': seed,
            #         'N': N,
            #         'sigma': sigma,
            #         'model': model,
            #         'sparsity': sparsity,
            #         'n_eval': y_test.shape[0],
            #         'rmse': rmses[i],
            #         'rmse_orig': rmses[i] * y_std
            #     }
            #     if compute_nll:
            #        row["nll"] = gaussian_nll_pointwise(y_test, y_hats[i], sig_s[i]).mean()
            #    results.append(row)

#    return pd.DataFrame(results), pd.DataFrame(posterior_means)
    return pd.DataFrame(posterior_means)


In [12]:
from utils.sparsity import forward_pass_relu, forward_pass_tanh, local_prune_weights

def build_masks_cache_for_all(
    all_fits,
    dataset_keys,
    models,
    sparsity_levels,
    prune_W2=False,
    method="Eabs_stability"
):
    masks_cache = {}
    for dataset_key in dataset_keys:
        for model in models:
            try:
                masks_cache[(dataset_key, model)] = precompute_global_masks(
                    all_fits=all_fits,
                    dataset_key=dataset_key,
                    model=model,
                    sparsity_levels=sparsity_levels,
                    prune_W2=prune_W2,
                    method=method
                )
            except KeyError:
                print(f"[SKIP MASKS] Missing fit for {dataset_key} -> {model}")
    return masks_cache

N100_seeds = [1, 3, 4, 5, 6]
N200_seeds = [2, 7, 8, 9, 10]
N500_seeds = [11, 12, 13, 14, 15]
all_seeds = [N100_seeds, N200_seeds, N500_seeds]
seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]


def get_N_sigma(seed, all_seeds):
    if seed in all_seeds[0]:
        N=100
    elif seed in all_seeds[1]:
        N=200
    else:
        N=500
    sigma=1.00
    return N, sigma
dataset_keys = []
for seed in seeds:
    N, sigma = get_N_sigma(seed, all_seeds)
    dataset_keys.append(f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}')


In [13]:
dataset_keys_nodewise = []
for seed in seeds:
    N, sigma = get_N_sigma(seed, all_seeds)
    dataset_keys_nodewise.append(f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}')

# Precompute masks once
masks_tanh_nodewise = build_masks_cache_for_all(tanh_fits_nodewise, dataset_keys_nodewise, model_names_tanh_nodewise, sparsity_levels, prune_W2=False)

In [14]:
df_post_tanh_nodewise = {}
for q in sparsity_levels:
    df_post_tanh_nodewise[q] = compute_sparse_metrics_results_globalmask_large_eval(
        seeds=seeds,
        all_seeds=all_seeds,
        models=model_names_tanh_nodewise,
        all_fits=tanh_fits_nodewise,
        get_N_sigma=get_N_sigma,
        forward_pass=forward_pass_tanh,
        folder="friedman",
        sparsity=q,
        masks_cache=masks_tanh_nodewise,
        prune_W2=False,
        compute_nll=True,
        noise_var_name="sigma",
        n_eval=1000,
        D=10,
        standardize_y=True,
        gen_uncorr=generate_Friedman_data_v2,
        gen_corr=None,
    )


In [15]:
df_post_tanh_full_nodewise = pd.concat(
    [df.assign(sparsity=sparsity) for sparsity, df in df_post_tanh_nodewise.items()],
    ignore_index=True
)

df_post_tanh_full_nodewise_agg = aggregate_over_seeds(df_post_tanh_full_nodewise, "posterior_mean_rmse_orig")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# -----------------------------
# USER SETTINGS
# -----------------------------
colors = {
    "Gaussian tanh": "C0",
    "Regularized Horseshoe tanh": "C1",
    "Dirichlet Horseshoe tanh": "C2",
    "Dirichlet Horseshoe tanh nodewise": "C2",
    "Dirichlet Student T tanh": "C3",
    "Dirichlet Student T tanh nodewise": "C3",
    "Beta Horseshoe tanh": "C4",
    "Beta Horseshoe tanh nodewise": "C4",
    "Beta Student T tanh": "C5",
    "Beta Student T tanh nodewise": "C5",
}

model_abbr = {
    "Gaussian tanh": "Gauss",
    "Regularized Horseshoe tanh": "RHS",
    "Dirichlet Horseshoe tanh": "DHS",
    "Dirichlet Horseshoe tanh nodewise": "DHS",
    "Dirichlet Student T tanh": "DST",
    "Dirichlet Student T tanh nodewise": "DST",
    "Beta Horseshoe tanh": "C4",
    "Beta Horseshoe tanh nodewise": "C4",
    "Beta Student T tanh": "C5",
    "Beta Student T tanh nodewise": "C5",
}

sparsity_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
rmse_col = "posterior_mean_rmse_orig"

Ns = [100, 200, 500]

# Provide your dataframes here
dfs = [(df_post_tanh_full_nodewise_agg_old, "Prune per sample"), (df_post_tanh_full_nodewise_agg, "Posterior prune")]

# Models to skip (same logic as your example; edit freely)
SKIP_MODELS = {
    #"Gaussian tanh",
    "Dirichlet Student T tanh",
    #"Dirichlet Student T tanh nodewise",
    "Beta Student T tanh",
    "Beta Student T tanh nodewise",
}
# -----------------------------
# PLOT
# -----------------------------
fig, axes = plt.subplots(2, 3, figsize=(14, 7), sharex=True, sharey=True)

for r, (df_src, row_title) in enumerate(dfs):
    # Filter upfront
    df_src = df_src.copy()
    df_src = df_src[df_src["sparsity"].isin(sparsity_levels)]
    df_src["sparsity"] = pd.Categorical(df_src["sparsity"], categories=sparsity_levels, ordered=True)

    # Aggregate across seeds
    #df_agg = aggregate_over_seeds(df_src, metric_col=rmse_col, agg_fun=AGG_FUN)

    for c, N in enumerate(Ns):
        ax = axes[r, c]
        dN = df_src[df_src["N"] == N].copy()
        dN["sparsity"] = pd.Categorical(dN["sparsity"], categories=sparsity_levels, ordered=True)

        for model, g in dN.groupby("model", sort=False):
            if model in SKIP_MODELS:
                continue
            if model not in colors:
                # fallback if a new model label appears
                colors[model] = "C7"

            g = g.sort_values("sparsity")

            # lw, alpha, ls, mk = (1.3, 0.7, "--", "v") if "nodewise" in model else (2.2, 1.0, "-", "o")
            lw, alpha, ls, mk = (2.2, 1.0, "-", "o")

            # main line
            ax.plot(
                g["sparsity"].astype(float),
                g["center"],
                lw=lw,
                alpha=alpha,
                ls=ls,
                marker=mk,
                color=colors[model],
                label=model_abbr[model],
            )

        ax.set_title(f"N={N}" if r == 0 else "", fontsize=12)
        ax.grid(True, linestyle="--", linewidth=0.5)
        ax.set_xticks(sparsity_levels)
        ax.tick_params(axis='both', which='major', labelsize=12)

        if c == 0:
            ax.set_ylabel(f"{row_title}\nRMSE", fontsize=14)
        if r == 1:
            ax.set_xlabel("Sparsity", fontsize=12)

# --- one legend for the whole figure (deduped) ---
handles, labels = axes[0, 0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))

# Put legends where you like (your original had two legends)
# axes[1, 1].legend(
#     by_label.values(),
#     by_label.keys(),
#     loc="upper left",
#     frameon=False,
# )
# axes[0, 1].legend(
#     by_label.values(),
#     by_label.keys(),
#     loc="upper left",
#     frameon=False,
# )

handles, labels = plt.gca().get_legend_handles_labels()
order = [2,3,0,1]
axes[0, 1].legend([handles[idx] for idx in order],[labels[idx] for idx in order],
                  loc="upper left",frameon=False, fontsize=12)
axes[1, 1].legend([handles[idx] for idx in order],[labels[idx] for idx in order],
                  loc="upper left",frameon=False, fontsize=12)
# axes[1, 0].set_xticks(fontsize=14)
# axes[1, 1].set_xticks(fontsize=14)
# axes[1, 2].set_xticks(fontsize=14)
plt.tight_layout(rect=[0, 0, 0.85, 1])
plt.savefig("figures_for_use_in_paper/friedman_sparsity_tanh.pdf", bbox_inches="tight")
plt.show()
